In [ ]:
!pip install -q biopython tqdm transformers tokenizers accelerate

import pandas as pd
import numpy as np
import torch
import random
from tqdm.auto import tqdm
from Bio.SeqUtils.ProtParam import ProteinAnalysis

# Repro + device
seed = 42
random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
device = "cuda" if torch.cuda.is_available() else "cpu"

print("Device:", device)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 38.1 MB/s eta 0:00:00
Device: cuda


In [ ]:
# Properties to evaluate
traits = ["charge_pH8","gravy","aromaticity","instability_index","mol_weight","iso_point"]

# Which traits should be steered DOWN (everything else = UP) — kept for parity
steer_down_traits = {"aromaticity", "instability_index"}

def is_steer_down(prop: str) -> bool:
    return prop in steer_down_traits

def get_score(seq: str, trait_name: str):
    """Compute Biopython-based property for a protein sequence."""
    try:
        pa = ProteinAnalysis(seq)
        fns = {
            "charge_pH8":        lambda x: x.charge_at_pH(7.0),
            "gravy":             lambda x: x.gravy(),
            "aromaticity":       lambda x: x.aromaticity(),
            "instability_index": lambda x: x.instability_index(),
            "mol_weight":        lambda x: x.molecular_weight(),
            "iso_point":         lambda x: x.isoelectric_point(),
        }
        return fns[trait_name](pa)
    except Exception:
        return np.nan

In [ ]:
from transformers import AutoModelForCausalLM
from tokenizers import Tokenizer
import torch, torch.nn.functional as F

MODEL_NAME = "hugohrban/progen2-large"
REV = "main"  # optionally pin to a commit hash for reproducibility

# Prefer bfloat16 on H200/A100-class GPUs; fall back to float16 if needed
dtype = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float16
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load WITHOUT device_map; then .to(device)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    torch_dtype=dtype,
    low_cpu_mem_usage=True,
    revision=REV,
).eval().to(device)

# Tokenizer from `tokenizers` lib (as per model card)
tokenizer = Tokenizer.from_pretrained(MODEL_NAME, revision=REV)
tokenizer.no_padding()

AA = set("ACDEFGHIKLMNPQRSTVWY")

print(f"Loaded: {MODEL_NAME} on {device} (dtype={dtype})")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

configuration_progen.py: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


modeling_progen.py: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/577M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Loaded: hugohrban/progen2-large on cuda (dtype=torch.bfloat16)


In [ ]:
from transformers import AutoModelForCausalLM
from tokenizers import Tokenizer
import torch, torch.nn.functional as F

MODEL_NAME = "hugohrban/progen2-large"
REV = "main"  # optionally pin to a commit hash for reproducibility

# Prefer bfloat16 on H200/A100-class GPUs; fall back to float16 if needed
dtype = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float16
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load WITHOUT device_map; then .to(device)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    torch_dtype=dtype,
    low_cpu_mem_usage=True,
    revision=REV,
).eval().to(device)

# Tokenizer from `tokenizers` lib (as per model card)
tokenizer = Tokenizer.from_pretrained(MODEL_NAME, revision=REV)
tokenizer.no_padding()

AA = set("ACDEFGHIKLMNPQRSTVWY")

print(f"Loaded: {MODEL_NAME} on {device} (dtype={dtype})")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded: hugohrban/progen2-large on cuda (dtype=torch.bfloat16)


In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from tqdm.auto import tqdm
from Bio.SeqUtils.ProtParam import ProteinAnalysis

# ----- Settings -----
n_sequences = 200          # total sequences to generate
sequence_length = 100      # amino acids per sequence
temperature = 0.7
top_k = 3
seed = 42

# Protein properties to score
traits = ["charge_pH7", "gravy", "aromaticity", "instability_index", "mol_weight", "iso_point"]

# ----- Reproducibility -----
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
device = "cuda" if torch.cuda.is_available() else "cpu"

# ----- Load Model & Tokenizer -----
from transformers import AutoModelForCausalLM
from tokenizers import Tokenizer

MODEL_NAME = "hugohrban/progen2-large"
REV = "main"
dtype = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float16

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    torch_dtype=dtype,
    low_cpu_mem_usage=True,
    revision=REV,
).eval().to(device)

tokenizer = Tokenizer.from_pretrained(MODEL_NAME, revision=REV)
tokenizer.no_padding()

AA = set("ACDEFGHIKLMNPQRSTVWY")

# ----- Generation Function -----
def top_k_sample(logits: torch.Tensor, k: int = 0, temperature: float = 1.0) -> int:
    """Temperature + optional top-k sampling; returns sampled token id (int)."""
    if temperature <= 0:
        return torch.argmax(logits, dim=-1).item()
    logits = logits / temperature
    if k and k > 0:
        topk = torch.topk(logits, k)
        probs = torch.nn.functional.softmax(topk.values, dim=-1)
        idx_in_topk = torch.multinomial(probs, num_samples=1).item()
        return topk.indices[idx_in_topk].item()
    else:
        probs = torch.nn.functional.softmax(logits, dim=-1)
        return torch.multinomial(probs, num_samples=1).item()

def generate_sequence_progen2(
    model,
    tokenizer,
    length: int = 100,
    temperature: float = 0.7,
    top_k: int = 3,
    max_steps: int = 2000,
) -> str:
    model_device = next(model.parameters()).device
    prompt = "1"
    ids = tokenizer.encode(prompt).ids
    input_ids = torch.tensor(ids, dtype=torch.long, device=model_device)
    seq_chars = []
    with torch.no_grad():
        for _ in range(max_steps):
            logits = model(input_ids).logits[-1, :]
            next_id = top_k_sample(logits, k=top_k, temperature=temperature)
            input_ids = torch.cat([input_ids, torch.tensor([next_id], device=model_device)], dim=0)
            tok = tokenizer.id_to_token(next_id)
            if tok == "2":
                break
            if len(tok) == 1 and tok in AA:
                seq_chars.append(tok)
                if len(seq_chars) >= length:
                    break
    return "".join(seq_chars)

# ----- Scoring Function -----
def get_score(seq: str, trait_name: str):
    try:
        pa = ProteinAnalysis(seq)
        fns = {
            "charge_pH7":        lambda x: x.charge_at_pH(7.0),
            "gravy":             lambda x: x.gravy(),
            "aromaticity":       lambda x: x.aromaticity(),
            "instability_index": lambda x: x.instability_index(),
            "mol_weight":        lambda x: x.molecular_weight(),
            "iso_point":         lambda x: x.isoelectric_point(),
        }
        return fns[trait_name](pa)
    except Exception:
        return np.nan

# ----- Generation and Scoring Loop -----
all_sequences = []
all_scores = []

for i in tqdm(range(n_sequences), desc="Generating sequences"):
    seq = generate_sequence_progen2(
        model=model,
        tokenizer=tokenizer,
        length=sequence_length,
        temperature=temperature,
        top_k=top_k,
    )
    trait_scores = {trait: get_score(seq, trait) for trait in traits}
    all_sequences.append(seq)
    all_scores.append(trait_scores)

# Save all sequences and scores
df = pd.DataFrame({'sequence': all_sequences, **{trait:[score[trait] for score in all_scores] for trait in traits}})
df.to_csv("progen2_200_sequences_scores.csv", index=False)
print("Saved all 200 sequences and scores to progen2_200_sequences_scores.csv")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generating sequences:   0%|          | 0/200 [00:00<?, ?it/s]

Saved all 200 sequences and scores to progen2_200_sequences_scores.csv


In [ ]:
import pandas as pd
import numpy as np

# Properties to evaluate
traits = ["charge_pH7","gravy","aromaticity","instability_index","mol_weight","iso_point"]

# Load the CSV
df = pd.read_csv("progen2_200_sequences_scores.csv")

for trait in traits:
    vals = df[trait].dropna().astype(float)
    mean = vals.mean()
    std = vals.std(ddof=1)
    n = len(vals)
    ci95 = 1.96 * (std / np.sqrt(n)) if n > 1 else np.nan
    direction = "DOWN" if trait in {"aromaticity", "instability_index"} else "UP"
    print(f"{trait:18s} ({direction}): mean={mean:.4f} | std={std:.4f} | 95% CI=±{ci95:.4f} | n={n}")

charge_pH7         (UP): mean=-0.4224 | std=3.2994 | 95% CI=±0.4619 | n=196
gravy              (UP): mean=-0.1808 | std=0.8492 | 95% CI=±0.1189 | n=196
aromaticity        (DOWN): mean=0.0236 | std=0.0367 | 95% CI=±0.0051 | n=196
instability_index  (DOWN): mean=99.4730 | std=71.1382 | 95% CI=±9.9594 | n=196
mol_weight         (UP): mean=8994.6477 | std=2097.8866 | 95% CI=±290.7522 | n=200
iso_point          (UP): mean=6.0813 | std=1.7428 | 95% CI=±0.2440 | n=196
